In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout


In [2]:
df = pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
### Here we drop   ""BloodPressure"" And ""SkinThickness""

In [7]:
df.drop(columns=['BloodPressure','SkinThickness'], inplace=True)

In [8]:
df.head()

,Pregnancies,Glucose,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,0,33.6,0.627,50,1
1,1,85,0,26.6,0.351,31,0
2,8,183,0,23.3,0.672,32,1
3,1,89,94,28.1,0.167,21,0
4,0,137,168,43.1,2.288,33,1


In [9]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [10]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

In [11]:
X_scaled.shape

(768, 6)

In [12]:
y.shape

(768,)

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 6), (154, 6), (614,), (154,))

In [15]:
model = Sequential()

model.add(Dense(32, activation='relu', input_dim = 6))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

d:\admin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.fit(X_train, y_train, batch_size=2, epochs=25, validation_data=(X_test, y_test))

Epoch 1/25
307/307 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7001 - loss: 0.5805 - val_accuracy: 0.7078 - val_loss: 0.5640
Epoch 2/25
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7575 - loss: 0.4871 - val_accuracy: 0.7143 - val_loss: 0.5439
Epoch 3/25
307/307 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7710 - loss: 0.4647 - val_accuracy: 0.7208 - val_loss: 0.5374
Epoch 4/25
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7674 - loss: 0.4521 - val_accuracy: 0.7143 - val_loss: 0.5349
Epoch 5/25
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7880 - loss: 0.4318 - val_accuracy: 0.7208 - val_loss: 0.5310
Epoch 6/25
307/307 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7898 - loss: 0.4508 - val_accuracy: 0.7208 - val_loss: 0.5303
Epoch 7/25
307/307 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7863 - loss: 0.4745 - val_accuracy: 0.7338 - val_loss: 0.5287
Epoch 8/25
307/307 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7960 - loss: 0.4267 - val_accuracy: 0.

### Keras Tuner: All-in-One Hyperparameter Tuning (Layers, Units, Activations, Optimizers, Dropout)

In [17]:
import kerastuner as kt

D:\Temp\ipykernel_17072\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [19]:
def build_model_optimizer(hp):
    model = Sequential()

    ### Now tune the number of hidden layers
    num_layers = hp.Int('num_layers', min_value=1, max_value = 20, step = 2)

    for i in range(num_layers):

        ## Now tune the Units(Node) per layer
        units = hp.Int('units_' + str(i), min_value=8, max_value=128, step=8)

        ## Tune for the activation function
        activation = hp.Choice('activation_' + str(i), values = ['relu', 'tanh', 'sigmoid', 'selu', 'elu'])

        ## for first layer
        if i == 0:
            model.add(Dense(units=units,activation=activation, input_dim=6))
        ## And the hidden layers
        else:
            model.add(Dense(units=units, activation=activation))

        ### Tune for the dropout
        dropout_rate = hp.Choice('DropOut_' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='sigmoid')) ####Thiis is output layer

    ## Now tune for the optimizer
    optimizer = hp.Choice('optimizers', values=['Adam', 'RMSprop', 'SGD', 'Adadelta'])

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [21]:
tuner_final = kt.RandomSearch(
    build_model_optimizer, 
    objective='val_accuracy',
    max_trials=10
)

d:\admin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
### Run and Search
tuner_final.search(X_train,y_train, epochs=50, validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 18s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 03m 49s


In [24]:
## Get the best Parameters
best_final_params = tuner_final.get_best_hyperparameters(num_trials=1)[0].values
print("Best Parameters :::::::")
for key,value in best_final_params.items():
    print(f" {key} : {value}")

Best Parameters :::::::
 num_layers : 1
 units_0 : 112
 activation_0 : relu
 DropOut_0 : 0.3
 optimizers : SGD
 units_1 : 112
 activation_1 : tanh
 DropOut_1 : 0.2
 units_2 : 56
 activation_2 : elu
 DropOut_2 : 0.7
 units_3 : 48
 activation_3 : selu
 DropOut_3 : 0.7
 units_4 : 64
 activation_4 : tanh
 DropOut_4 : 0.6
 units_5 : 64
 activation_5 : selu
 DropOut_5 : 0.4
 units_6 : 56
 activation_6 : sigmoid
 DropOut_6 : 0.3
 units_7 : 80
 activation_7 : elu
 DropOut_7 : 0.9
 units_8 : 48
 activation_8 : elu
 DropOut_8 : 0.5
 units_9 : 56
 activation_9 : elu
 DropOut_9 : 0.6
 units_10 : 104
 activation_10 : tanh
 DropOut_10 : 0.5
 units_11 : 16
 activation_11 : tanh
 DropOut_11 : 0.8
 units_12 : 56
 activation_12 : relu
 DropOut_12 : 0.4
 units_13 : 128
 activation_13 : sigmoid
 DropOut_13 : 0.7
 units_14 : 96
 activation_14 : selu
 DropOut_14 : 0.1
 units_15 : 8
 activation_15 : selu
 DropOut_15 : 0.3
 units_16 : 88
 activation_16 : elu
 DropOut_16 : 0.4
 units_17 : 88
 activation_17 : s

In [29]:
### Get the best model
best_model_final = tuner_final.get_best_models(num_models=1)

In [30]:
best_model_final[0].fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.7456 - loss: 0.5123 - val_accuracy: 0.7987 - val_loss: 0.5264
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7882 - loss: 0.4745 - val_accuracy: 0.7987 - val_loss: 0.5240
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7605 - loss: 0.4949 - val_accuracy: 0.7922 - val_loss: 0.5215
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7458 - loss: 0.5012 - val_accuracy: 0.7922 - val_loss: 0.5194
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7777 - loss: 0.4774 - val_accuracy: 0.7922 - val_loss: 0.5178
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7328 - loss: 0.4892 - val_accuracy: 0.7987 - val_loss: 0.5161
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7668 - loss: 0.4958 - val_accuracy: 0.7987 - val_loss: 0.5148
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7378 - loss: 0.5064 - val_accuracy: 0.7